# Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

#### Color Printing

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

#print(color.BOLD + 'Hello World !' + color.END)

### Label Encoding

In [3]:
le = preprocessing.LabelEncoder()

# data_process(dataset_path) 
#### performs the data input & aggelos' processing.
# Preparexy(dataframe)
#### performs data cleaning & preparing.

In [4]:
def preparexy(df):
    # Data Preparing
    # Encode labels with value 0-> n_classes -1
    
    features = ['I50', 'Φ50', 'I150', 'Φ150', 'I250', 'Φ250']
    
    # Features - X
    X = df[features]

    # Calculate Z-score in order to find outliers
    z = np.abs(stats.zscore(X))
    #print(z) # Visualize
    threshold = 2.5 # Change the threshold arbitrarily
    #print(np.where(z > threshold))
    df = df[(z < threshold).all(axis=1)] # Remove outliers that exceed the threshold given from dataset

    # Now get as X the 'clean' features
    X = df[features]

    # Scale the inputs (4 options; put comment in the ones you are not using)

    # -1- Standard Scaling
    #scaler = StandardScaler()
    #X = scaler.fit_transform(X)

    # -2- Min-max scaling 
    #scaler = preprocessing.MinMaxScaler()
    #X = scaler.fit_transform(X)

    # -3- Robust scaling
    scaler = preprocessing.RobustScaler()
    X = scaler.fit_transform(X)

    # -4- Normalizing
    #scaler = preprocessing.Normalizer()
    #X = scaler.fit_transform(X)

    # Label - Y
    y = df['appliance']

    # Convert y to integer 
    le.fit(y) # Fit label encoder
    y = le.transform(y) # Transform labels to normalized encoding
    
    return(X,y)

def data_process(dataset_path):
    df = pd.read_excel(dataset_path)

    # Data Cleaning with Aggelos Rules for 50, 150, 250 phases
    df = df[(df.I50 > 0.1) & (df.I150 > 0.01) & (df.I250 > 0.01)] # Clean useless current features


    #df['Φ50'] = df['Φ50'].apply(pd.to_numeric)

    # For angle between (90, 180):
    # Modify by +180 degrees
    rows_with_rads_to_decrease = df.loc[(df['Φ50'] > 90) & (df['Φ50'] < 180)]
    rows_with_rads_to_decrease['Φ50'] -= 180
    df.update(rows_with_rads_to_decrease)

    # For angle between (-180, -90):
    # Modify by -180 degrees
    rows_with_rads_to_increase = (df.loc[(df['Φ50'] < -90) & (df['Φ50'] > -180)])
    rows_with_rads_to_increase['Φ50'] += 180
    df.update(rows_with_rads_to_increase)
    
    return df

### Load Datasets 
Following, you can either:
- train on house1, test on house2
- or split train/test set on houseX

In [5]:
# Uncomment the following if you want to train on house1 and test on house2:
#X_train, y_train = preparexy(data_process("../datasets/appliances_combination_daskio.xls"))
#X_test, y_test = preparexy(data_process("../datasets/appliances_combination_veroia.xls"))

######----#####------######----#####------######----#####------######----#####------

# Train & Testing in 1 dataset only
X, y = preparexy(data_process("../datasets/appliances_combination_daskio.xls"))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C:\Users\kaslou\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kaslou\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## printMetrics method (Used when splitting train/test so you have a $X_{test}$ and a $y_{test}$)
Takes as input the $classifier$, the $X_{test}$ and the $y_{test}$.

Then it prints a detailed report with all the known metrics. 

In [6]:
def printMetrics(clf, X_test, y_test):
    # Uncomment if you want confusion matrix to be shown
    #cm = confusion_matrix(y_test, y_pred) 
    #print("Confusion matrix")
    #print(cm)
    
    
    # This will print precision, recall, f1-score, support for all the categories
    #target_names = ['class 0', 'class 1', 'class 2']
    print("Classification report for classifier \n%s:\n%s" % (clf, metrics.classification_report(y_test, y_pred)))
    print("Accuracy: %1.3f" % clf.score(X_test, y_test))
    print("-----------------\n")   

# get_metrics Method
### Takes as input the true class and the predicted class
#### Then, splits these classes into the labels containing each class and computes Precision, Recall, Support

In [7]:
# Compute the metrics mathematically 
def get_metrics(true_string, predicted_string):
    
    true_array = true_string.split("+") # Find substrings and split them
    predicted_array = predicted_string.split("+")
    
    # Find which elements of array A are in array B
    comparePositives = np.in1d(predicted_array, true_array)
    compareNegatives = np.in1d(true_array, predicted_array)

    #print(comparePositives)
    #print(compareNegatives)
    
    TP = TN = FP = FN = 0 # Initialiaze the metrics
    
    # Scan the arrays for each elements' presence
    for predictedLabel in comparePositives:
        if (predictedLabel == True):
            TP = TP + 1
        if (predictedLabel == False):
            FP = FP + 1
            
    for predictedLabel in compareNegatives:
        if (predictedLabel == True):
            TN = TN + 1
        if (predictedLabel == False):
            FN = FN + 1
            
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    #F1 = 2*((precision*recall)/(precision+recall))
    F1 = 20.231231
    
    #print (precision, recall, F1)
    
    return precision, recall, F1

#print(get_metrics("DVD-TV+Laptop+Pistolaki", "DVD-TV+Tostiera"))

## Decision Trees
##### Uncomment and then add the for-loop (2 cells below) in order to run it

In [8]:
# Decision Trees
#clf = DecisionTreeClassifier(random_state = 42) # Feel free to change 'min_samples_split' 
#clf.fit(X_train, y_train)

#print("Decision Trees:")
#y_pred = clf.predict(X_test)

#printMetrics(clf, X_test, y_test)

## Multi-layer Perceptron with 18 neurons in 1 hidden layer

In [9]:
#print(X_test.shape, y_test.shape)
#print(le.inverse_transform(y_test[330])) # Check one TRUE class 

#true = le.inverse_transform(y_test[330])
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(18,), random_state=42)
#clf = LogisticRegression()
clf.fit(X_train, y_train)

print("Multi-layer Perceptron:")
y_pred = clf.predict(X_test)
#print(le.inverse_transform(y_pred[330])) # check one PREDICTED class
#predicted = le.inverse_transform(y_pred[330])

#printMetrics(clf, X_test, y_test)

Multi-layer Perceptron:


In [10]:
for i in range(0, len(y_test)):

    true_class = le.inverse_transform(y_test[i])
    predicted_class = le.inverse_transform(y_pred[i])
    #print(y_test[i], y_pred[i])
    #print('Actual class is {}{} and the classifier predicted {}'.format(color.BOLD, true_class, predicted_class))

    print('Actual class is ' + color.BOLD  + color.BLUE+ true_class + color.END +' and the classifier predicted ' + color.BOLD + color.RED + predicted_class + color.END)
    precision, recall, F1 = get_metrics(true_class, predicted_class)
    
    print('Precision is {0:.4f}, Recall is {0:.4f} and F1-Score is {0:.4f}'.format(precision, recall, F1))
    
    print('\n')
    
    
    

Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Pistolaki+Mati and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Laptop and the classifier predicted DVD-TV+Laptop
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is Laptop+mpriki+Tostiera and the classifier predicted Laptop+mpriki+Tostiera
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Psigio+Mati and the classifier predicted DVD-TV+Mati
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is Laptop+Psigio+Tostiera and the classifier predicted Laptop+Psigio+Tostiera
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Mati and the classifier predicted DVD-TV+Mati
Precision is 1.0000, Recall is 1.0000 and F1-Score

C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b


Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Pistolaki+Mati and the classifier predicted DVD-TV+Pistolaki
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki+Tostiera
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Psigio and the classifier predicted DVD-TV+Tostiera
Precision is 0.5000, Recall is 0.5000 and F1-Score is 0.5000


Actual class is Laptop+Pistolaki+Tostiera and the classifier predicted Psigio+Pistolaki+Tostiera
Precision is 0.6667, Recall is 0.6667 and F1-Score is 0.6667


Actual class is DVD-TV+Psigio+Pistolaki and the classifier predicted DVD-TV+Psigio+Pistolaki
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Pistol

C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b



Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is Laptop+Psigio+Pistolaki and the classifier predicted Laptop+Psigio+Pistolaki
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Pistolaki and the classifier predicted DVD-TV+Pistolaki
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Tostiera and the classifier predicted DVD-TV+Tostiera
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is Laptop+Pistolaki+Tostiera and the classifier predicted Psigio+Pistolaki+Tostiera
Precision is 0.6667, Recall is 0.6667 and F1-Score is 0.6667


Actual class is DVD-TV+Pistolaki and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 0.6667, Recall is 0.6667 and F1-Score is 0.6667


Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0000, Rec

...

C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

Precision is 0.6667, Recall is 0.6667 and F1-Score is 0.6667


Actual class is DVD-TV+Pistolaki+Mati and the classifier predicted DVD-TV+Pistolaki+Mati
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is Psigio+Pistolaki+Mati and the classifier predicted Psigio+Pistolaki+Mati
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is Laptop+Pistolaki and the classifier predicted Laptop+Pistolaki
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Pistolaki and the classifier predicted DVD-TV+Pistolaki
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is Psigio+Pistolaki+Tostiera and the classifier predicted Psigio+Pistolaki+Tostiera
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is Laptop+mpriki+Tostiera and the classifier predicted Laptop+mpriki+Tostiera
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Pistolaki an

C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

Actual class is DVD-TV+Laptop+Mati and the classifier predicted DVD-TV+Laptop+Mati
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Psigio+Pistolaki and the classifier predicted DVD-TV+Psigio+Pistolaki
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is Laptop+mpriki+Tostiera and the classifier predicted Laptop+mpriki+Tostiera
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Pistolaki+Tostiera and the classifier predicted DVD-TV+Pistolaki
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Psigio+Tostiera and the classifier predicted DVD-TV+Tostiera
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Psigio+Pistolaki and the classifier predicted DVD-TV+Psigio+Pistolaki
Precision is 1.0000, Recall is 1.0000 and F1-Score is 1.0000


Actual class is DVD-TV+Psigio+Pistolaki and the classifier predicted DVD-TV+Psigio+Pistola

C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\kaslou\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b